# Narupa Multiplayer Selections Test

This notebook demonstrates how to create a selection via the multiplayer key value store.

In [ ]:
from narupa.app.client import NarupaImdClient

## Start a Narupa IMD client

In [ ]:
# Setup a Narupa Client to automatically find a server on the network (server needs to be discoverable)
client = NarupaImdClient.autoconnect()
first_frame = client.wait_until_first_frame(check_interval=0.5, timeout=10)
print(first_frame.particle_count)

In [ ]:
# Connect to the multiplayer
client.subscribe_multiplayer()

## Example Snippets

### Create a new selection

In [ ]:
# Create a selection called 'Selection' which selects particles with indices 0-4
selection = client.create_selection("Selection", [0, 1, 2, 3, 4])

### Delete all selections

In [ ]:
# Remove all selections from the system
client.clear_selections()

### Add more indices to the selection

In [ ]:
# Add the particles with indices 3-5 to the selection, without changing the old selection
with selection.modify():
    selection.add_particles([5, 4, 3])

### Print the current selection(s)

In [ ]:
# Print out the name and set of particle indices in each selection
for selection in client.selections:
    print(selection.selection_name, selection.selected_particle_ids)

### Replace the selection indices

In [ ]:
# Replace the selection so it only contains the particles with indices 1-3
with selection.modify():
    selection.set_particles([1,2,3])

### Clear the selection

In [ ]:
# Clear a single selection, which means it is considered to contains all particles
with selection.modify():
    selection.clear_particles()

### Set a selection from MDAnalysis

In [ ]:
# Helper function to generate selections using the MDAnalysis selection language

from narupa.mdanalysis import frame_data_to_mdanalysis

def generate_mdanalysis_selection(selection: str):
    universe = frame_data_to_mdanalysis(client.first_frame)
    idx_array = universe.select_atoms(selection).indices
    return map(int, idx_array)

### Set the rendering of a selection

In [ ]:
# Set the renderer for a selection

with selection.modify():
    selection.renderer = {
        'color': 'CornflowerBlue',
        'render': 'goodsell'
    }

### Get a gradient from matplotlib

In [ ]:
# Helper function to get a gradient from matplotlib.

import matplotlib.cm

def get_matplotlib_gradient(name: str):
    cmap = matplotlib.cm.get_cmap(name)
    return list(list(cmap(x/7)) for x in range(0, 8, 1))

## Example Setup - Neuraminidase

In [ ]:
# Hide root selection
root_selection = client.root_selection
with root_selection.modify():
    root_selection.hide = True
    root_selection.interaction_method = 'none'

In [ ]:
# Select protein without hydrogens
protein = client.create_selection("Protein", [])
with protein.modify():
    protein.set_particles(generate_mdanalysis_selection("protein and not type H"))

In [ ]:
# Select ligand
ligand = client.create_selection("Ligand", [])
with ligand.modify():
    ligand.set_particles(generate_mdanalysis_selection("resname OSE"))

In [ ]:
# Render protein
with protein.modify():
    protein.renderer = {
            'color': {
                'type': 'residue index in entity',
                'gradient': get_matplotlib_gradient('viridis')
            },
            'scale': 0.2,
            'render': 'cycles'
        }
    protein.interaction_method = 'none'

In [ ]:
# Render ligand
with ligand.modify():
    ligand.renderer = {
            'color': 'IndianRed',
            'scale': 0.1,
            'render': 'liquorice'
        }
    ligand.velocity_reset = True
    ligand.interaction_method = 'group'